Draw a Sankey diagram!

Load the data created by the script `scripts/assemble_flows.py`:

In [ ]:
import pandas as pd
flows = pd.read_csv('data/flows.csv')

Define the Sankey diagram structure and draw:

In [ ]:
from palettable.cartocolors.qualitative import Prism_10, Antique_10
C = Prism_10.hex_colors[1:]

In [ ]:
from floweaver import *

sectors = ['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15a', '15b', '16', '17', '18', '19', '20']

manufac_and_products = [
    (sector, ['sector %s' % sector, 'products %s' % sector])
    for sector in sectors
]

semis = ['Hot rolled', 'Plate', 'Cold rolled', 'Hot dipped galvanised',
       'Electro coated', 'Organic coated', 'Tin plate', 'Tubes and pipes',
       'Railway track material', 'Sheet piling and rolled accessories',
        'Heavy sections', 'Light sections',
       'Hot rolled bars in lengths', 'Bright bars', 'Reinforcing bar', 'Rods']

products = [('Structural', ['Structural steelwork and building and civil engineering']),
       'Mechanical engineering', 'Vehicles',
       'Electrical engineering', 'Shipbuilding',
       'Boilers, drums and other vessels','Metal goods', 
       'Other industries']

psemi = Partition.Simple('material', semis)
pprod = Partition.Simple('material', sectors)

pmat = Partition.Simple('material', [
    ('Flat', ['Hot rolled', 'Plate', 'Cold rolled', 'Hot dipped galvanised',
              'Electro coated', 'Organic coated', 'Tin plate', 'Tubes and pipes']),
    ('Long', ['Railway track material', 'Sheet piling and rolled accessories', 'Heavy sections', 'Light sections',
              'Hot rolled bars in lengths', 'Bright bars', 'Reinforcing bar', 'Rods']),
    ('Metal goods', ['4', '5', '6', '7']),
    ('Elec. eng', ['8']),
    ('Mech. eng', ['9', '10', '11', '12', '13']),
    ('Construction', ['14']),
    ('Transport', ['15a', '15b', '16']),
    ('Energy/water', ['17', '18', '19']),
    ('Other', ['20']),
    'component',
    ('scrap', ['scrap %s' % p for p in semis])
])

nodes = {
    'prod': ProcessGroup(['uk_production'], title='UK production'),
    'semi': ProcessGroup(['uk_intermediate'], psemi, title='Semi'),
    'manuf': ProcessGroup([yy for xx in manufac_and_products for yy in xx[1]],
                          Partition.Simple('process', manufac_and_products), title='Manufacturing'),
    #'products': ProcessGroup(['products'], pprod, title='Products'),
    'demand': ProcessGroup(['uk_demand'], pprod, title='UK demand'),
    
    'scrap': ProcessGroup(['scrap'], title='Manufacturing scrap'),
    
    'semi_export': ProcessGroup(['exports'], title='Export of semis'),
    'semi_import': ProcessGroup(['imports'], title='Import'),
    'comp_import': ProcessGroup(['component_imports'], title='Import components'),
    'prod_export': ProcessGroup(['product_exports'], title='Export of goods'),
    'prod_import': ProcessGroup(['product_imports'], title='Import of steel-containing goods'),
    'imbalance': ProcessGroup(['imbalance'])
}

ordering = [
    [['semi_import'], ['prod'], []],
    [['comp_import', 'imbalance'], ['semi'], ['semi_export']],
    [['prod_import'], ['manuf']],
    [[], ['demand'], ['prod_export', 'scrap']],
]

bundles = [
    Bundle('prod', 'semi'),
    Bundle('prod', 'semi_export'),
    Bundle('semi_import', 'semi'),
    Bundle('semi', 'manuf'),
    
    Bundle('comp_import', 'manuf'),
    #Bundle('manuf', 'products'),
    Bundle('manuf', 'scrap'),
    Bundle('manuf', 'demand'),
    Bundle('manuf', 'prod_export'),
    Bundle('prod_import', 'demand'),
#     Bundle('products', 'demand'),
#     Bundle('products', 'prod_export'),
    
    #Bundle('imbalance', 'products'),
    Bundle('imbalance', 'manuf'),
]

palette = {k: '#999' for k in pmat.labels}
palette['Flat'] = Antique_10.hex_colors[0]
palette['Long'] = Antique_10.hex_colors[2]
palette['Metal goods'] = C[0]
palette['Elec. eng'] = C[1]
palette['Mech. eng'] = C[2]
palette['Construction'] = C[3]
palette['Transport'] = C[4]
palette['Energy/water'] = C[5]
palette['Other'] = C[6]
palette['component'] = Antique_10.hex_colors[1]

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=pmat)
weave(sdd, flows.query('year == "2009"'), palette=palette) \
.to_widget(width=1200, height=1000, debugging=True, align_link_types=False)
#.auto_save_svg('figures/sankey_2016.svg')

In [ ]:
for year in range(2009, 2017):
    weave(sdd, flows.query('year == @year'), palette=palette) \
          .to_json('figures/sankey_%d.json' % year)